# 論文を読む
# Faster R-CNN: Towards Real-Time Object
# Detection with Region Proposal Networks


## (1) 物体検出の分野にはどういった手法が存在したか。
Advances like SPPnet [1] and Fast R-CNN [2] have reduced the running time of these detection networks, exposing region proposal computation as a bottleneck.

Fast R-CNN とSPPnetという手法があることがわかる。


### Fast R-CNN

R-CNN/SPPnetでは、fine-tune/classification/bounding box regressionをそれぞれ別々に学習する必要がありました。そこで、Fast R-CNNではこれらを課題を解決するために、下記の手法を提案しました

RoI pooling layerという、SPPのpyramid構造を取り除いたシンプルな幅可変poolingを行う
classification/bounding box regressionを同時に学習させるためのmulti-task lossによって1回で学習ができるようにする(ad hocでない)
オンラインで教師データを生成する工夫
またmulti-task lossの導入により、Back-Propagationが全層に適用できるようになったため、全ての層の学習が可能になりました。

### SPPnet

この論文のアイディアとしては、画像をリサイズするのではなく、feature mapsをpoolingするときにリサイズしてしまおう、というものです。

R-CNNでは、固定サイズの画像を入力として識別していましたが、 SPPnetでは、Spatial Pyramid Pooling (SPP)という手法を用いることで、 CNNで畳み込んだ最終層のfeature mapsを縦横可変サイズで取り扱えるようにしました。

これにより、R-CNNのように大量の物体量領域ごとにCNNで特徴抽出するのでなく(約2000個のRegion Proposalはかなり領域の重複が多いため、重複する画像領域をCNNで特徴抽出するのはかなり無駄)、 画像１枚から大きなfeature mapsを作成した後、Region Proposalの領域の特徴をSPPによってベクトル化することで、スピードはGPU上にて24-102倍に高速化できました。


There is a large literature on object
proposal methods. Comprehensive surveys and comparisons of object proposal methods can be found in
[19], [20], [21]. Widely used object proposal methods
include those based on grouping super-pixels (e.g.,
Selective Search [4], CPMC [22], MCG [23]) and those
based on sliding windows (e.g., objectness in windows
[24], EdgeBoxes [6]). Object proposal methods were
adopted as external modules independent of the detectors (e.g., Selective Search [4] object detectors, RCNN [5], and Fast R-CNN [2]).


### Selective Search
色やテクスチャ特徴を基に類似度が高い近接領域を段階ごとに結合していき、最終的に一枚の画像を一つの結合体にします。その段階的な変化を基に物体領域を認識するという方法。精度が高く、処理速度も高速ですがパラメータ調整が大変なようです。複数のパラメータ設定で並列処理することで認識率を上げることができます。物体の輪郭も得る事ができますがCPMCにの精度には及びません。物体の詳細な輪郭を得る必要がない場合は良い選択肢の一つになりそうです。

### CPMC
古くからある物体認識方法です。前景として配置された点(Seeds)の周囲にグラフカット処理が行われ、それらを基に物体領域を認識するという方法。処理速度が非常に遅いがかなりの高精度で物体領域が検出できるようです。また強みとしてObjectnessでは得られない物体の輪郭が高精度で得られるという点があり、目的の物体を背景から切り離して処理したい時などに適しています。

### MCG
Multiscale Combinatorial Grouping  
領域分割計算を高速に行うために、求めた領域から効率よく物体候補を算出するGroupingアルゴリズムの提案で、 画像ピラミッドを生成し、いろんなスケールでの領域分割を行い、別々に一般的な手法で領域分割を行って、「だいたいこのへんが境界だろう」という情報を統合してから、詳細な分割を行う。
その中心となるのは、正規化カットセグメンテーションのための高速固有ベクトル計算と、階層領域の組み合わせマージのための効率的なアルゴリズムです。

### EdgeBoxes 
物体検出の計算効率を高めるため。エッジを用いてオブジェクト境界ボックスの提案を生成。
ボックス内に存在するエッジの数からボックスの境界に重なる輪郭のメンバーであるものを差し引いた単純なボックス客観性スコアを提案。

## (2) Fasterとあるが、どういった仕組みで高速化したのか。
Advances like SPPnet [1] and Fast R-CNN [2] have reduced the running time of these detection networks, exposing region
proposal computation as a bottleneck. In this work, we introduce a Region Proposal Network (RPN) that shares full-image
convolutional features with the detection network, thus enabling nearly cost-free region proposals. An RPN is a fully convolutional
network that simultaneously predicts object bounds and objectness scores at each position

既存の手法ではRegion Proposalの計算がボトルネックになっている。この研究では、全画像のたたみ込み機能を検出ネットワークと共有するためのRegion Proposal Network（RPN）を導入して、計算コストの低いのRegion Proposalを可能にします。

## (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

One-Stage Detection vs. Two-Stage Proposal + Detection. The OverFeat paper [9] proposes a detection
method that uses regressors and classifiers on sliding
windows over convolutional feature maps. OverFeat
is a one-stage, class-specific detection pipeline, and ours
is a two-stage cascade consisting of class-agnostic proposals and class-specific detections. In OverFeat, the
region-wise features come from a sliding window of
one aspect ratio over a scale pyramid. These features
are used to simultaneously determine the location and
category of objects. In RPN, the features are from
square (3×3) sliding windows and predict proposals
relative to anchors with different scales and aspect
ratios. Though both methods use sliding windows, the
region proposal task is only the first stage of Faster RCNN—the downstream Fast R-CNN detector attends
to the proposals to refine them. In the second stage of
our cascade, the region-wise features are adaptively
pooled [1], [2] from proposal boxes that more faithfully cover the features of the regions. We believe
these features lead to more accurate detections.

 One-Stageの手法であるOverFeatは1段階のクラス固有の検出パイプラインであり、Two-StageのFaster R-CNNはクラスにとらわれないproposesとクラス固有の検出からなる2段階のカスケードです。
 OverFeatでは、Regionごとのfeatureは、スケールピラミッド上の縦横比が1のスライド式ウィンドウに由来します。
 オブジェクトの位置とカテゴリを同時に判断するために使用されます。オブジェクトの位置とカテゴリを同時に判断するために使用されます。
 
 RPNでは、特徴は正方形（3×3）のスライディングウィンドウからのものであり、異なる縮尺とアスペクト比を持つアンカーに対するproposalsを予測します。どちらの方法もスライディングウィンドウを使用しますが、Region Proposal タスクはより高速なRCNNの最初の段階にすぎません。ダウンストリームのFast R-CNN detectorは、proposalに注意を向けてそれらを洗練します。私たちのカスケードの第2段階では、region-wise featuresは、proposalのfeaturesをより忠実にカバーする proposal boxes から適応的にプールされます。

OverFeatについて：畳み込みネットワークを用いた統合認識、位置確認および検出
画像分類、物体位置特定および検出のための統一モデル。この論文では、両方のタスクに対して同時にモデルをトレーニングすると、画像分類と画像検出精度が向上すると主張しています。

OverFeatの3つの新しいアイデア：

画像内の複数の場所に、スライディングウィンドウ方式で、複数のスケールにわたってConvNetを適用します。しかし、これでも、多くの表示ウィンドウには、オブジェクトの完全に識別可能な部分（犬の頭など）が含まれている場合がありますが、オブジェクト全体ではなく、オブジェクトの中心でさえありません。これはまともな分類につながりますが、ローカライゼーションと検出が不十分になります。

したがって、第２のアイデアは、各ウィンドウについてカテゴリにわたる分布を生成するだけでなく、ウィンドウに対するオブジェクトを含む境界ボックスの位置およびサイズの予測を生成するようにシステムを訓練することである。

3番目のアイデアは、各場所とサイズで各カテゴリの証拠を蓄積することです。



## (4) RPNとは何か。
(3.1 Region Proposal Networks)A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score.より、Region Proposal Network（RPN）は（任意のサイズの）画像を入力として受け取り、それぞれが物体らしさのスコアを持つ1組の長方形のオブジェクト予測を出力すネットワークのことであり、 Because our ultimate goal is to share computation with a Fast R-CNN object detection network [2], we assume that both nets share a common set of convolutional layers.から最終的な目標はFastR-CNNオブジェクト検出ネットワーク[2]とRPNが計算を共有することであるので、両方のネットワークが共通のたたみ込み層を共有すると仮定している。

## (5)RoIプーリングとは何か
 The RoI pooling layer uses max pooling to convert the features inside any valid region of interest into a small feature map with a fixed spatial extent of H × W (e.g., 7 × 7), where H and W are layer hyper-parameters that are independent of any particular RoI. ＲｏＩプーリングレイヤは、マックスプーリングを使用して任意の有効なROIのfeature-mapをＨ×Ｗ（例えば７×７）の固定サイズのfeatur-mapに変換する。これによりR-CNNでは必要な入力前の画像のリサイズを行う必要がない.
 
 

## (6) Anchorのサイズはどうするのが適切か。
(3.1 Region Proposal Networks)To generate region proposals, we slide a small network over the convolutional feature map output by the last shared convolutional layer. At each sliding-window location, we simultaneously predict multiple region proposals, where the number of maximum possible proposals for each location is denoted as k 領域提案を生成するために、最後の共有たたみ込みレイヤによって出力されたたたみ込み特徴マップ上に小さなネットワークをスライドさせる。各スライディングウィンドウ位置で複数の領域提案を予測し、各位置に対する最大可能提案boxの数はkで表される。

By default we use 3 scales and 3 aspect ratios, yielding k=9 anchors at each sliding position.論文によるとアンカー数kはデフォルトでスケールごとに3種類、アスペクト比ごとに3種類のk=9を採用している

## (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか

We comprehensively evaluate our method on the PASCAL VOC 2007 detection benchmark 
ASCAL VOC 2007というデータセットを使っている。

We primarily evaluate detection mean Average Precision (mAP), because this is the actual metric for object detection (rather than focusing on object proposal proxy metrics). 精度の評価はmAPで行う。これは実際の物体検出の指標値なため。 Table X compares the two-stage system and two variants of the one-stage system. Using the ZF model, the one-stage system has an mAP of 53.9%. This is lower than the two-stage system (58.7%) by 4.8% One-stageのモデルのmAPが53.9%でFaster-RCNNは58.7%となり4.8%上回った

We also provide results on the PASCAL VOC 2012 benchmark for a few models.

いくつかのモデルではPASCAL VOC 2012のデータセットでも評価している

We present more results on the Microsoft COCO object detection dataset マイクロソフトのCOCOというデータセットを使用している We experiment with the 80k images on the training set, 40k images on the validation set, and 20k images on the test-dev set. 80000個のtrain, 40000個のvalidation, 20000個のtest

In Table XI we first report the results of the Fast R-CNN system [2] using the implementation in this paper. Our Fast R-CNN baseline has 39.3% mAP@0.5 on the test-dev set, higher than that reported in [2].Next we evaluate our Faster R-CNN system. Using the COCO training set to train, Faster R-CNN has 42.1% mAP@0.5 and 21.5% mAP@[.5, .95] on the COCO test-dev set. This is 2.8% higher for mAP@0.5 and 2.2% higher for mAP@[.5, .95] than the Fast R-CNN counterpart under the same protocol (Table XI) Fast R-CNNは原著論文で35.9のmAP,この論文の測定では39.7mAPであった。FasterR-CNNは 同じプロトコルの下でのFast R-CNN対応物よりも、mAP @ 0.5では2.8％、mAP @ [。5、.95]では2.2％高い結果であった